In [1]:
import pandas as pd
import numpy as np
import psycopg2 as pg
import geopandas as gpd
import networkx as nx
from tqdm.auto import tqdm 
tqdm.pandas()

/usr/local/lib/python3.8/dist-packages/geopandas/_compat.py:112: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [21]:
class Provision:

    def __init__(self, SUB_GROUP:str='schools'):
        self.SUB_GROUP = SUB_GROUP
        # self.g = None

    def prepare_graph(self):
        # Тут нужно включить впн
        engine = pg.connect("dbname='city_db_final' user='postgres' host='10.32.1.107' port='5432' password='postgres'")
        # GROUPS = ('Детский сад', 'Школа', 'Рекреационная зона')
        # SUB_GROUP = GROUPS[0]
        matrix = pd.read_pickle('/home/gk/jupyter/MMG_SPB.pkl')
        blocks_geom = gpd.read_parquet('/home/gk/jupyter/masterplanning/output_data/blocks.parquet')
        buildings = gpd.read_parquet('/home/gk/jupyter/masterplanning/output_data/buildings.parquet')
        res = gpd.sjoin(blocks_geom, buildings, predicate='intersects', how='left').drop(columns=['id_right', 'index_right']).groupby('id_left').agg(
                        {'population_balanced': 'sum',
                        'living_area': 'sum'})
        
        res.reset_index(drop=False, inplace=True)
        res['living_area'] = res['living_area'].apply(lambda x: True if x > 0 else False)
        res.rename(columns={'living_area': 'is_living'}, inplace=True)
        blocks_geom = res.merge(blocks_geom, right_on='id', left_on='id_left')
        blocks_geom.drop(columns=['id_left'], inplace=True)
        living_blocks = blocks_geom.loc[:, ['id', 'geometry']].sort_values(by='id').reset_index(drop=True)

        service_blocks_df = gpd.read_postgis(f" select capacity, geometry as geom from all_services where city_service_type_code in ('{self.SUB_GROUP}') "\
                        "and city_id=1", con=engine)
        
        service_blocks_df.rename(columns={'geom': 'geometry'}, inplace=True)

        service_blocks_df["geometry"] = service_blocks_df["geometry"].convex_hull
        service_blocks_df["geometry"] = service_blocks_df["geometry"].to_crs(32636)
        service_blocks_df["geometry"] = service_blocks_df["geometry"].centroid

        blocks_geom = gpd.GeoDataFrame(blocks_geom, geometry='geometry', crs=32636)

        service_blocks_df["geometry"] = service_blocks_df["geometry"].set_crs(32636)
        service_blocks_df = service_blocks_df.set_geometry("geometry")

        res2 = gpd.sjoin(blocks_geom, service_blocks_df, predicate='intersects').groupby('id').agg(
            {
            'capacity': 'sum',
            }
        )
        service_blocks_df = res2.copy()

        blocks_geom_dict = blocks_geom[['id', 'population_balanced', 'is_living']].set_index('id').to_dict()
        service_blocks_dict = service_blocks_df.to_dict()['capacity']

        blocks_list = matrix.loc[matrix.index.isin(service_blocks_df.index.astype('Int64')),
                            matrix.columns.isin(living_blocks['id'])]

        ACCS_TIME = 30
        self.g = nx.Graph()

        for idx in tqdm(list(blocks_list.index)):

            blocks_list_tmp = blocks_list[blocks_list.index == idx]
            blocks_list.columns = blocks_list.columns.astype(int)
            blocks_list_tmp = blocks_list_tmp[blocks_list_tmp < ACCS_TIME].dropna(axis=1)
            blocks_list_tmp_dict = blocks_list_tmp.transpose().to_dict()[idx]
            
            for key in blocks_list_tmp_dict.keys():

                if key != idx:
                    self.g.add_edge(idx, key, weight=round(blocks_list_tmp_dict[key], 1))

                else:
                    
                    self.g.add_node(idx)

                self.g.nodes[key]['population'] = blocks_geom_dict['population_balanced'][int(key)]
                self.g.nodes[key]['is_living'] = blocks_geom_dict['is_living'][int(key)]

                if  key != idx:
                    try:
                        if self.g.nodes[key][f'is_{self.SUB_GROUP}_service'] != 1:
                            self.g.nodes[key][f'is_{self.SUB_GROUP}_service'] = 0
                            self.g.nodes[key][f'provision_{self.SUB_GROUP}'] = 0
                            self.g.nodes[key][f'id_{self.SUB_GROUP}'] = 0
                    except KeyError:
                        self.g.nodes[key][f'is_{self.SUB_GROUP}_service'] = 0
                        self.g.nodes[key][f'provision_{self.SUB_GROUP}'] = 0
                        self.g.nodes[key][f'id_{self.SUB_GROUP}'] = 0
                else:
                    self.g.nodes[key][f'is_{self.SUB_GROUP}_service'] = 1
                    self.g.nodes[key][f'{self.SUB_GROUP}_capacity'] = service_blocks_dict[key]
                    self.g.nodes[key][f'provision_{self.SUB_GROUP}'] = 0
                    self.g.nodes[key][f'id_{self.SUB_GROUP}'] = 0

                if self.g.nodes[key]['is_living'] == True:
                    self.g.nodes[key][f'population_prov_{self.SUB_GROUP}'] = 0
                    self.g.nodes[key][f'population_unprov_{self.SUB_GROUP}'] = 0
                
    
    def get_stats(self):
        g = self.g
        if not g:
            return 0
        
        blocks_service = 0 
        blocks_bad =0
        blocks_living = 0
        total = 0
        for key in g:
            total +=1
            try:
                if  g.nodes[key]['is_living'] == True:
                    blocks_living +=1
                elif  g.nodes[key][f'is_{self.SUB_GROUP}_service'] == 1:
                    blocks_service +=1
            except KeyError as ex:
                blocks_bad += 1

        print(f'количество кварталов c сервисом {self.SUB_GROUP}: {blocks_service}')  
        print(f'количество жилых кварталов: {blocks_living}') 
        print(f'количество кварталов всего: {total}')
        print(f'количество кварталов c ошибкой: {blocks_bad}')


    def get_provision(self):
        g = self.g
        for node in g.nodes:
            if g.nodes[node][f'is_{self.SUB_GROUP}_service'] == 1:
                capacity = g.nodes[node][f'{self.SUB_GROUP}_capacity'] 
                neighbors = list(g.neighbors(node))
                for neighbor in neighbors:
                    if g.nodes[neighbor]['is_living'] == True and g.nodes[neighbor]['population'] > 0:
                        population = g.nodes[neighbor]['population']
                        if self.SUB_GROUP == 'schools':
                            load = (population / 1000) * 120
                        elif self.SUB_GROUP == 'kindergartens':
                            load = (population / 1000) * 60
                        elif self.SUB_GROUP == 'recreational_areas':
                            load = population 
                        if  g.nodes[neighbor][f'provision_{self.SUB_GROUP}'] == 0 and load <= capacity: 
                            prov_proc = (population * 100) / population
                            g.nodes[neighbor][f'provision_{self.SUB_GROUP}'] += prov_proc
                            capacity -= load
                            g.nodes[neighbor][f'id_{self.SUB_GROUP}'] = node
                            g.nodes[neighbor][f'population_prov_{self.SUB_GROUP}'] += population
                            g.nodes[neighbor][f'population_unprov_{self.SUB_GROUP}'] = 0
                        elif g.nodes[neighbor][f'provision_{self.SUB_GROUP}'] == 0 and load > capacity:
                            if capacity > 0:
                                if self.SUB_GROUP == 'schools':
                                    prov_people = (capacity * 1000) / 120
                                elif self.SUB_GROUP == 'kindergartens':
                                    prov_people = (capacity * 1000) / 60
                                elif self.SUB_GROUP == 'recreational_areas':
                                    prov_people =  capacity     
                                unprov_people = population - prov_people
                                prov_pop = (prov_people  * 100) / population
                                capacity -= capacity
                                g.nodes[neighbor][f'population_prov_{self.SUB_GROUP}'] += prov_people
                                g.nodes[neighbor][f'population_unprov_{self.SUB_GROUP}'] += unprov_people
                                g.nodes[neighbor][f'id_{self.SUB_GROUP}'] = node
                                g.nodes[neighbor][f'provision_{self.SUB_GROUP}'] += prov_pop
                        else:
                            g.nodes[neighbor][f'provision_{self.SUB_GROUP}'] += 0
        self.g = g
    
    def get_geo(self, blocks):
        g = self.g
        blocks[f'provision_{self.SUB_GROUP}'] = 0
        blocks[f'id_{self.SUB_GROUP}'] = 0
        blocks[f'population_prov_{self.SUB_GROUP}'] = 0 
        blocks[f'population_unprov_{self.SUB_GROUP}'] = 0
        blocks[f'provision_{self.SUB_GROUP}'] = 0
        for n in g:
            indx = blocks[blocks ['id'] == n].index[0]
            if g.nodes[n]['is_living'] == True:
                if g.nodes[n].get(f'id_{self.SUB_GROUP}') is not None:
                    blocks.loc[indx, f'id_{self.SUB_GROUP}'] = g.nodes[n][f'id_{self.SUB_GROUP}']
                    blocks.loc[indx, f'population_prov_{self.SUB_GROUP}'] = g.nodes[n][f'population_prov_{self.SUB_GROUP}']
                    blocks.loc[indx, f'population_unprov_{self.SUB_GROUP}'] = g.nodes[n][f'population_unprov_{self.SUB_GROUP}']
                    blocks.loc[indx, f'provision_{self.SUB_GROUP}'] = g.nodes[n][f'provision_{self.SUB_GROUP}']
                else:
                    blocks[f'population_unprov_{self.SUB_GROUP}'][indx] = g.nodes[n][f'population_unprov_{self.SUB_GROUP}']

        blocks[f'id_{self.SUB_GROUP}'] = blocks[f'id_{self.SUB_GROUP}'].astype(int)
        blocks[f'population_prov_{self.SUB_GROUP}'] =  blocks[f'population_prov_{self.SUB_GROUP}'].astype(int)
        blocks[f'population_unprov_{self.SUB_GROUP}'] = blocks[f'population_unprov_{self.SUB_GROUP}'].astype(int)
        blocks[f'provision_{self.SUB_GROUP}'] = blocks[f'provision_{self.SUB_GROUP}'].astype(int)

        return blocks


In [22]:
import os

GROUPS = ('kindergartens', 'schools', 'recreational_areas')
blocks = gpd.read_parquet('/home/gk/jupyter/masterplanning/output_data/blocks.parquet')

for sub_group in GROUPS:

    # if not os.path.exists(f'../output_data/{sub_group}.parquet'):
    model = Provision(SUB_GROUP=sub_group)
    model.prepare_graph()
    model.get_stats()
    model.get_provision()
    gdf = model.get_geo(blocks)
    gdf.to_parquet(f'../output_data/{sub_group}.parquet')


/usr/local/lib/python3.8/dist-packages/geopandas/io/sql.py:154: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(
100%|██████████| 595/595 [00:10<00:00, 58.41it/s]


количество кварталов c сервисом kindergartens: 23
количество жилых кварталов: 1808
количество кварталов всего: 5605
количество кварталов c ошибкой: 0


/usr/local/lib/python3.8/dist-packages/geopandas/io/sql.py:154: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(
100%|██████████| 520/520 [00:09<00:00, 57.08it/s]


количество кварталов c сервисом schools: 42
количество жилых кварталов: 1806
количество кварталов всего: 5586
количество кварталов c ошибкой: 0


/usr/local/lib/python3.8/dist-packages/geopandas/io/sql.py:154: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(
100%|██████████| 2034/2034 [00:35<00:00, 56.69it/s]


количество кварталов c сервисом recreational_areas: 1073
количество жилых кварталов: 1814
количество кварталов всего: 5686
количество кварталов c ошибкой: 0
